In [1]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import re
import numpy as np
import pandas as pd
from keras.layers import Dense, Input, Bidirectional, GRU
from keras.layers import Embedding
from keras.preprocessing import text, sequence
from keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D, Reshape, Conv2D, MaxPool2D, Flatten, Dropout
from keras.models import Model

from tensorflow.keras.utils import to_categorical
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.losses import CategoricalCrossentropy, BinaryCrossentropy
import fasttext
from sklearn.model_selection import train_test_split
from keras_preprocessing.sequence import pad_sequences



In [ ]:
import re
def clean(data):
    with open('teencode.txt','r') as file:
      file = file.read()
      lines = file.split('\n')
      for line in lines:
        elements = line.split('\t')
        data = re.sub(r'\b{}\b'.format(elements[0]), elements[1], data)
    data = re.sub('per ','',data)
    data = re.sub(r'\s+', ' ', data)
    return data
  # print(s)

In [2]:
df_train= pd.read_excel('train_nor_811.xlsx')
df_test= pd.read_excel('test_nor_811.xlsx')
df_valid= pd.read_excel('valid_nor_81.xlsx')


df_train['Sentence'] = df_train['Sentence'].apply(clean)
df_test['Sentence'] = df_test['Sentence'].apply(clean)
df_valid['Sentence'] = df_valid['Sentence'].apply(clean)

In [3]:
X_train= df_train.loc[:,'Sentence']
y_train= df_train.loc[:,'Emotion']
enc1 = LabelEncoder()
y_train = enc1.fit_transform(y_train)
y_train= pd.Series(y_train)

X_test= df_test.loc[:,'Sentence']
y_test= df_test.loc[:,'Emotion']
enc2 = LabelEncoder()
y_test = enc2.fit_transform(y_test)
y_test= pd.Series(y_test)

X_valid= df_valid.loc[:,'Sentence']
y_valid= df_valid.loc[:,'Emotion']
enc3 = LabelEncoder()
y_valid = enc3.fit_transform(y_valid)
y_valid= pd.Series(y_valid)

In [4]:


max_features = 2000
maxlen = 1000
embed_size = 300
batch_size = 1024
epochs = 10
tokenizer = text.Tokenizer(num_words=None, lower=False, filters='!"#$%&()*+,-./:;<=>?@[\\]^`{|}~\t\n')
tokenizer.fit_on_texts(X_train.astype(str))


In [5]:
embeddings_index ={}

#Dowload cc.vi.300 at: https://fasttext.cc/docs/en/crawl-vectors.html
with open('cc.vi.300.vec', encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs



In [6]:
word_index = tokenizer.word_index
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, embed_size))
max_features = num_words

for word, i in word_index.items():
    if i >= max_features:
        continue

    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [7]:


filter_sizes = [1,2,3,5]
num_filters = 32

inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights =[embedding_matrix])(inp)
x = SpatialDropout1D(0.4)(x)
x = Reshape((maxlen, embed_size, 1))(x)

conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embed_size), kernel_initializer='normal',
                activation='elu')(x)
conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embed_size), kernel_initializer='normal',
                activation='elu')(x)
conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embed_size), kernel_initializer='normal',
                activation='elu')(x)
conv_3 = Conv2D(num_filters, kernel_size=(filter_sizes[3], embed_size), kernel_initializer='normal',
                activation='elu' )(x)

maxpool_0 = MaxPool2D(pool_size=(maxlen - filter_sizes[0] + 1, 1))(conv_0)
maxpool_1 = MaxPool2D(pool_size=(maxlen - filter_sizes[1] + 1, 1))(conv_1)
maxpool_2 = MaxPool2D(pool_size=(maxlen - filter_sizes[2] + 1, 1))(conv_2)
maxpool_3 = MaxPool2D(pool_size=(maxlen - filter_sizes[3] + 1, 1))(conv_3)

z = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2, maxpool_3])
z = Flatten()(z)
# z = Dense(128, activation='relu')(z) #moi them
z = Dropout(0.4)(z)

outp = Dense(7, activation="softmax")(z)

optimizer = Adam(beta_1=0.9, beta_2=0.999, decay=0.0)

loss = CategoricalCrossentropy()

model = Model(inputs=inp, outputs=outp)
model.compile(loss=loss,
              optimizer=optimizer,
              metrics=['categorical_accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1000)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1000, 300)    1327200     input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d (SpatialDropo (None, 1000, 300)    0           embedding[0][0]                  
__________________________________________________________________________________________________
reshape (Reshape)               (None, 1000, 300, 1) 0           spatial_dropout1d[0][0]          
______________________________________________________________________________________________

2022-12-29 16:37:16.414832: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [8]:

X_train_fold = [str(p) for p in list(X_train)]
X_train_fold = tokenizer.texts_to_sequences(X_train_fold)
X_train_fold = pad_sequences(X_train_fold, maxlen=maxlen)

X_test_fold = [str(p) for p in list(X_test)]
X_test_fold = tokenizer.texts_to_sequences(X_test_fold)
X_test_fold = pad_sequences(X_test_fold, maxlen=maxlen)

X_valid_fold = [str(p) for p in list(X_valid)]
X_valid_fold = tokenizer.texts_to_sequences(X_valid_fold)
X_valid_fold = pad_sequences(X_valid_fold, maxlen=maxlen)

y_train_fold = to_categorical(y_train, num_classes=7)
y_valid_fold = to_categorical(y_valid, num_classes=7)
y_test_fold = y_test

callback = EarlyStopping(patience=3) #moi them
model.fit(x=X_train_fold, y=y_train_fold, validation_data=(X_valid_fold, y_valid_fold), batch_size=64, epochs=10, verbose=1, callbacks= [callback])
prediction = model.predict(X_test_fold, batch_size=batch_size, verbose=1)
test_pred = prediction.argmax(axis=-1)
evaluate = f1_score(y_test_fold, test_pred, average='weighted')
confuse = confusion_matrix(y_test_fold, test_pred, labels=[0, 1, 2, 3, 4, 5, 6])

2022-12-29 16:37:16.929908: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
87/87 [==============================] - 149s 2s/step - loss: 1.7670 - categorical_accuracy: 0.2882 - val_loss: 1.6025 - val_categorical_accuracy: 0.3878
Epoch 2/10
87/87 [==============================] - 147s 2s/step - loss: 1.5316 - categorical_accuracy: 0.4183 - val_loss: 1.4151 - val_categorical_accuracy: 0.4781
Epoch 3/10
87/87 [==============================] - 149s 2s/step - loss: 1.2911 - categorical_accuracy: 0.5406 - val_loss: 1.2802 - val_categorical_accuracy: 0.5277
Epoch 4/10
87/87 [==============================] - 150s 2s/step - loss: 1.0822 - categorical_accuracy: 0.6218 - val_loss: 1.1972 - val_categorical_accuracy: 0.5379
Epoch 5/10
87/87 [==============================] - 153s 2s/step - loss: 0.9183 - categorical_accuracy: 0.6878 - val_loss: 1.1590 - val_categorical_accuracy: 0.5583
Epoch 6/10
87/87 [==============================] - 150s 2s/step - loss: 0.7684 - categorical_accuracy: 0.7410 - val_loss: 1.1619 - val_categorical_accuracy: 0.5729
Epoch 7/10

In [9]:
print(classification_report(y_test_fold, test_pred,digits=4)) #0.56

              precision    recall  f1-score   support

           0     0.5556    0.3750    0.4478        40
           1     0.5479    0.6061    0.5755       132
           2     0.6418    0.6684    0.6548       193
           3     0.8333    0.6522    0.7317        46
           4     0.4333    0.5039    0.4659       129
           5     0.6364    0.5431    0.5860       116
           6     0.4706    0.4324    0.4507        37

    accuracy                         0.5743       693
   macro avg     0.5884    0.5401    0.5589       693
weighted avg     0.5828    0.5743    0.5753       693



In [10]:
model.save('model_cnn.h5')

In [ ]:
import pickle

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)